In [18]:
import pandas as pd
import numpy as np

from keras.models import load_model
from keras.utils import to_categorical
from scipy.stats import zscore

from os.path import join
from datetime import datetime

In [19]:
weather_path = join('..','data','weather','precip_temp.csv')
weather_df = pd.read_csv(weather_path)
# Add normalized columns
weather_df[['TMIN-norm','TMAX-norm']] = weather_df[['TMIN','TMAX']] - (weather_df[['TMIN', 'TMAX']].sum().sum() \
                              / weather_df[['TMIN', 'TMAX']].size)
weather_df[['TMIN-norm','TMAX-norm']] /= (weather_df[['TMIN-norm', 'TMAX-norm']].abs().sum().sum() \
                              / weather_df[['TMIN-norm', 'TMAX-norm']].size)
weather_df['PRCP-norm'] = zscore(weather_df['PRCP'])

model = load_model('model.h5')

In [20]:
def predict_at_date(date):
    """
    Estimate wifi usage on a given date
    
    Args:
        date: string in '%Y-%m-%d' format, e.g. '2017-05-30'
    
    Returns:
        dictionary with estimate for:
            # Clients
            # Sessions
            4 hour blocks of usage in bytes
        as well as day of the week and weather
    """
    dt = datetime.strptime(date, '%Y-%m-%d')
    weather = weather_df[weather_df['DATE'] == date][['PRCP', 'TMAX', 'TMIN']].values
    weather_norm = weather_df[weather_df['DATE'] == date] \
        [['PRCP-norm', 'TMAX-norm', 'TMIN-norm']].values
    date = np.array([dt.timetuple()[7] / 365])
    day = to_categorical(dt.weekday(), num_classes=7)
    predictions = model.predict([weather_norm, np.array([day]), date])
    return {'usage': predictions[0], '# clients': predictions[1],
            '# sessions': predictions[2], 'day': dt.strftime('%A'),
           'weather': {'precipitation':weather[0][0], 'high':weather[0][1], 'low':weather[0][2]}}

In [21]:
predict_at_date('2017-05-30')

{'# clients': array([[48.728317]], dtype=float32),
 '# sessions': array([[436.88882]], dtype=float32),
 'day': 'Tuesday',
 'usage': array([[15204.548 ,  3058.044 ,  6173.813 ,  2424.7856,  3680.3906,
         19471.607 ]], dtype=float32),
 'weather': {'high': 25.9, 'low': 16.1, 'precipitation': 0.0}}